In [1]:
import pandas as pd
import requests
import os
from geojson import FeatureCollection

##### IMPORT LOCAL DATA


In [3]:
bars = pd.read_excel("nashville-bars-data.xlsx")
MAPBOX_TOKEN = os.getenv("MAPBOX_TOKEN")

In [4]:
def mapboxGeocode(params: dict = {}):
    baseUrl = "https://api.mapbox.com/search/geocode/v6/forward"
    req = requests.get(baseUrl, params=params)
    return req.json()


def getGeocodeCoordinates(result: FeatureCollection):
    return ",".join(([str(x) for x in result['features'][0]["geometry"]["coordinates"]]))


def getAddressCoordinates(address):
    mapbox_query = {
        "q": address,
        "access_token": MAPBOX_TOKEN,
        "country": 'US'
    }
    geocode_results = mapboxGeocode(mapbox_query)
    string_coordinates = getGeocodeCoordinates(geocode_results)
    return string_coordinates

In [5]:
bars['coordinates'] = bars["address"].apply(lambda x: getAddressCoordinates(x))
bars['longitude'] = bars["coordinates"].apply(lambda x: float(x.split(",")[0]))
bars['latitude'] = bars["coordinates"].apply(lambda x: float(x.split(",")[1]))

In [6]:
bars.to_json("nashville-bars-data.json", orient="records")

In [7]:
pd.read_json("nashville-bars-data.json").to_csv("nashville-bars-data.csv")